# Pitch Monitor

Plot pitch angle (motors off).

In [10]:
import nest_asyncio
nest_asyncio.apply()

import asyncio, sys, os
sys.path.append(os.path.join(os.getenv('IOT49'), 'projects/robot/code-pi'))

from robot import RobotComm
from robot import LivePlot
import stm32
import numpy as np

from bokeh.io import output_notebook

output_notebook()

def exception_handler(loop, context):
    msg = context.get("exception", context["message"])
    print("***** asyncio:", context)
    print("***** msg:", msg)
    
async def main(fs=10, N=100):
    async with RobotComm(fs=fs) as rc:
        asyncio.get_event_loop().set_exception_handler(exception_handler)
        balance_angle = 6.5
        lp = LivePlot('time [s]', ['pitch [deg]'], rollover=200, title="Pitch Angle", y_range=(balance_angle-10, balance_angle+10))
        lp.show(rc.get_state_queue())
        try:
            rc.start_controller(fs, 'duty_control')
            rc.set_fstate('duty1', 0)
            rc.set_fstate('duty2', 0)
            await asyncio.sleep(60)
            await rc.ping()
            rc.stop_controller()
        finally:
            pass

print("reset")
stm32.hard_reset()
if False:
    print("rsync")
    stm32.rsync(dry_run=False)
    print("reset again")
    stm32.hard_reset()

asyncio.run(main(fs=10, N=100))


Loading BokehJS ...

reset
MCU: init controller



KeyboardInterrupt: 